In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import torch
import argparse
from models import EncoderCNN, SentenceLSTM, WordLSTM
from collections import Counter

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)


from dataloader import get_loader
from score import evalscores
from torchvision import transforms
from torch import nn
import numpy as np
from nltk.translate.bleu_score import corpus_bleu

[1000/6473] Tokenized the captions.
[2000/6473] Tokenized the captions.
[3000/6473] Tokenized the captions.
[4000/6473] Tokenized the captions.
[5000/6473] Tokenized the captions.
[6000/6473] Tokenized the captions.


In [2]:
from cococaption.pycocotools.coco import COCO
from cococaption.pycocoevalcap.eval import COCOEvalCap
import json

In [3]:
import pandas as pd

In [4]:
from collections import Counter

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)


In [5]:
!nvidia-smi

Tue May  5 17:46:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   51C    P0   225W / 300W |  14494MiB / 16155MiB |     47%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   54C    P0   268W / 300W |  13748MiB / 16158MiB |     90%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [6]:
from train import script

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--img_size', type = int, default = 224, help = 'size to which image is to be resized')
parser.add_argument('--crop_size', type = int, default = 224, help = 'size to which the image is to be cropped')
parser.add_argument('--device_number', type = str, default = "0", help = 'which GPU to run experiment on')


parser.add_argument('--int_stop_dim', type = int, default = 64, help = 'intermediate state dimension of stop vector network')
parser.add_argument('--sent_hidden_dim', type = int, default = 512, help = 'hidden state dimension of sentence LSTM')
parser.add_argument('--sent_input_dim', type = int, default = 1024, help = 'dimension of input to sentence LSTM')
parser.add_argument('--word_hidden_dim', type = int, default = 512, help = 'hidden state dimension of word LSTM')
parser.add_argument('--word_input_dim', type = int, default = 512, help = 'dimension of input to word LSTM')
parser.add_argument('--att_dim', type = int, default = 64, help = 'dimension of intermediate state in co-attention network')
parser.add_argument('--num_layers', type = int, default = 1, help = 'number of layers in word LSTM')


parser.add_argument('--lambda_sent', type = int, default = 1, help = 'weight for cross-entropy loss of stop vectors from sentence LSTM')    
parser.add_argument('--lambda_word', type = int, default = 1, help = 'weight for cross-entropy loss of words predicted from word LSTM with target words')


parser.add_argument('--batch_size', type = int, default = 8, help = 'size of the batch')
parser.add_argument('--shuffle', type = bool, default = True, help = 'shuffle the instances in dataset')
parser.add_argument('--num_workers', type = int, default = 0, help = 'number of workers for the dataloader')
parser.add_argument('--num_epochs', type = int, default = 8, help = 'number of epochs to train the model')
parser.add_argument('--learning_rate_cnn', type = int, default = 1e-5, help = 'learning rate for CNN Encoder')
parser.add_argument('--learning_rate_lstm', type = int, default = 5e-3, help = 'learning rate for LSTM Decoder')


parser.add_argument('--log_step', type=int , default=10, help='step size for prining log info')
parser.add_argument('--save_step', type=int , default=1000, help='step size for saving trained models')

_StoreAction(option_strings=['--save_step'], dest='save_step', nargs=None, const=None, default=1000, type=<class 'int'>, choices=None, help='step size for saving trained models', metavar=None)

In [8]:
args = parser.parse_args('')

In [9]:
# os.environ["CUDA_VISIBLE_DEVICES"]= args.device_number
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(args.device)

cuda


In [10]:
args

Namespace(att_dim=64, batch_size=8, crop_size=224, device=device(type='cuda'), device_number='0', img_size=224, int_stop_dim=64, lambda_sent=1, lambda_word=1, learning_rate_cnn=1e-05, learning_rate_lstm=0.005, log_step=10, num_epochs=8, num_layers=1, num_workers=0, save_step=1000, sent_hidden_dim=512, sent_input_dim=1024, shuffle=True, word_hidden_dim=512, word_input_dim=512)

In [11]:
args, val_loader, encoderCNN, sentLSTM, wordLSTM, vocab = script(args)

vocab_size:  743
<start>
[['the cardiomediastinal silhouette is normal in size and contour.', 'no focal consolidation pneumothorax or large pleural effusion.', 'negative for acute displaced rib fracture.', 'no discrete xxxx <unk> visualized.', 'contrast within the bilateral renal <unk> <unk>.', 'contrast also probably within the left colon.']]
[]
Bleu_1: 0.000
Bleu_2: 0.000
Bleu_3: 0.000
Bleu_4: 0.000
ROUGE_L: 0.000
CIDEr: 0.000
Epoch [0/8], Step [0/810], Loss: 54.6027
Epoch [0/8], Step [10/810], Loss: 7.1921
Epoch [0/8], Step [20/810], Loss: 7.3709
Epoch [0/8], Step [30/810], Loss: 7.4790
Epoch [0/8], Step [40/810], Loss: 6.0623
Epoch [0/8], Step [50/810], Loss: 4.7547
Epoch [0/8], Step [60/810], Loss: 6.1365
Epoch [0/8], Step [70/810], Loss: 3.6706
Epoch [0/8], Step [80/810], Loss: 4.0334
Epoch [0/8], Step [90/810], Loss: 5.2055
Epoch [0/8], Step [100/810], Loss: 4.9751
Epoch [0/8], Step [110/810], Loss: 5.6967
Epoch [0/8], Step [120/810], Loss: 5.8410
Epoch [0/8], Step [130/810], Lo

Epoch [2/8], Step [80/810], Loss: 3.5116
Epoch [2/8], Step [90/810], Loss: 3.1679
Epoch [2/8], Step [100/810], Loss: 2.7711
Epoch [2/8], Step [110/810], Loss: 3.3052
Epoch [2/8], Step [120/810], Loss: 3.7961
Epoch [2/8], Step [130/810], Loss: 3.2859
Epoch [2/8], Step [140/810], Loss: 4.7169
Epoch [2/8], Step [150/810], Loss: 2.7369
Epoch [2/8], Step [160/810], Loss: 3.8772
Epoch [2/8], Step [170/810], Loss: 2.9956
Epoch [2/8], Step [180/810], Loss: 4.7573
Epoch [2/8], Step [190/810], Loss: 2.3957
Epoch [2/8], Step [200/810], Loss: 2.2857
Epoch [2/8], Step [210/810], Loss: 4.4098
Epoch [2/8], Step [220/810], Loss: 4.7864
Epoch [2/8], Step [230/810], Loss: 3.5988
Epoch [2/8], Step [240/810], Loss: 2.0672
Epoch [2/8], Step [250/810], Loss: 3.0875
Epoch [2/8], Step [260/810], Loss: 4.4701
Epoch [2/8], Step [270/810], Loss: 4.8708
Epoch [2/8], Step [280/810], Loss: 3.3186
Epoch [2/8], Step [290/810], Loss: 2.5407
Epoch [2/8], Step [300/810], Loss: 2.9657
Epoch [2/8], Step [310/810], Loss: 3

Epoch [4/8], Step [150/810], Loss: 2.9120
Epoch [4/8], Step [160/810], Loss: 3.4075
Epoch [4/8], Step [170/810], Loss: 4.3114
Epoch [4/8], Step [180/810], Loss: 3.1094
Epoch [4/8], Step [190/810], Loss: 3.6887
Epoch [4/8], Step [200/810], Loss: 3.5351
Epoch [4/8], Step [210/810], Loss: 3.2751
Epoch [4/8], Step [220/810], Loss: 4.9880
Epoch [4/8], Step [230/810], Loss: 2.5801
Epoch [4/8], Step [240/810], Loss: 3.8611
Epoch [4/8], Step [250/810], Loss: 4.9627
Epoch [4/8], Step [260/810], Loss: 2.3901
Epoch [4/8], Step [270/810], Loss: 2.5259
Epoch [4/8], Step [280/810], Loss: 3.5273
Epoch [4/8], Step [290/810], Loss: 2.9043
Epoch [4/8], Step [300/810], Loss: 3.6338
Epoch [4/8], Step [310/810], Loss: 2.9178
Epoch [4/8], Step [320/810], Loss: 3.0541
Epoch [4/8], Step [330/810], Loss: 2.6960
Epoch [4/8], Step [340/810], Loss: 4.1922
Epoch [4/8], Step [350/810], Loss: 3.3126
Epoch [4/8], Step [360/810], Loss: 3.5890
Epoch [4/8], Step [370/810], Loss: 3.5961
Epoch [4/8], Step [380/810], Loss:

Epoch [6/8], Step [240/810], Loss: 3.3974
Epoch [6/8], Step [250/810], Loss: 2.7997
Epoch [6/8], Step [260/810], Loss: 1.9445
Epoch [6/8], Step [270/810], Loss: 3.0264
Epoch [6/8], Step [280/810], Loss: 2.5551
Epoch [6/8], Step [290/810], Loss: 3.3555
Epoch [6/8], Step [300/810], Loss: 3.3480
Epoch [6/8], Step [310/810], Loss: 3.1951
Epoch [6/8], Step [320/810], Loss: 2.8458
Epoch [6/8], Step [330/810], Loss: 3.6322
Epoch [6/8], Step [340/810], Loss: 2.4903
Epoch [6/8], Step [350/810], Loss: 3.2708
Epoch [6/8], Step [360/810], Loss: 2.7056
Epoch [6/8], Step [370/810], Loss: 4.4311
Epoch [6/8], Step [380/810], Loss: 3.1007
Epoch [6/8], Step [390/810], Loss: 1.9987
Epoch [6/8], Step [400/810], Loss: 3.3862
Epoch [6/8], Step [410/810], Loss: 3.0097
Epoch [6/8], Step [420/810], Loss: 2.8608
Epoch [6/8], Step [430/810], Loss: 2.4789
Epoch [6/8], Step [440/810], Loss: 2.4673
Epoch [6/8], Step [450/810], Loss: 3.7502
Epoch [6/8], Step [460/810], Loss: 3.6793
Epoch [6/8], Step [470/810], Loss:

### Теперь протестируем нашу модель на просто каком-то одном батче и посмотрим что выходит

In [12]:
encoderCNN

EncoderCNN(
  (cnn): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [13]:
encoderCNN.eval()
sentLSTM.eval()
wordLSTM.eval()

vocab_size = len(vocab)

In [14]:
for i, (images, captions, prob) in enumerate(val_loader):
    break

In [15]:
images.shape

torch.Size([8, 3, 224, 224])

In [16]:
captions.shape

torch.Size([8, 6, 17])

In [17]:
captions[:3, :, :10]

tensor([[[  1,  38, 110,   9, 109,   2,   0,   0,   0,   0],
         [  1,  15, 201,   2,   0,   0,   0,   0,   0,   0],
         [  1,  53,   6, 120,   9,  11,   2,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],

        [[  1,  23,  13,   6,  18, 155, 196,  15,  20,  57],
         [  1,  53,  30,   6,   7, 103,  10,  11,  12,   2],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],

        [[  1,  11,  53,   2,   0,   0,   0,   0,   0,   0],
         [  1,  14,  13,   2,   0,   0,   0,   0,   0,   0],
         [  1,  87,  44,  45,  37, 188,   2,   0,   0,   0],
         [  1,  15,  16,   2,   0,   0,   0,   0,   0,   0],
         [  1,  15, 

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
images = images.to(args.device)
captions = captions.to(args.device)
prob = prob.to(args.device)

In [20]:
vis_enc_out = encoderCNN(images)

topics, ps = sentLSTM(vis_enc_out, torch.ones_like(captions), args.device)

# loss_sent = criterion_stop_val(ps.view(-1, 2), prob.view(-1))

# loss_word = torch.tensor([0.0]).to(args.device)

pred_words = torch.zeros((captions.shape[0], captions.shape[1], captions.shape[2])).to(args.device)

In [21]:
topics[:2, 0, :20]

tensor([[ 0.0000,  0.0000, 22.0881, 17.5690, 10.0925,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.6939,  0.9649,  0.0000, 18.9178,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, 51.1835, 18.9786,  0.0000, 19.4284,  0.0000,  5.0696,  0.9998,
          0.0000,  0.0000,  0.7694,  0.0000,  0.1152,  1.4709,  0.6992, 13.1206,
          0.0000,  0.0000,  0.0000,  0.0000]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [22]:
captions[:, 0, :2]

tensor([[ 1, 38],
        [ 1, 23],
        [ 1, 11],
        [ 1, 53],
        [ 1, 23],
        [ 1, 23],
        [ 1, 23],
        [ 1, 11]], device='cuda:0')

In [23]:
pred_words[:, :, 0] = 1
pred_words = pred_words.long()
pred_words.shape

torch.Size([8, 6, 17])

In [24]:
for i in range(captions.shape[1]):
    for j in range(2, captions.shape[2]):
        pred_words[:, i, :j] = wordLSTM(topics[:, i, :], pred_words[:, i, :j]).argmax(2)

In [25]:
pred_words[:, 0, :]

tensor([[ 1, 23, 53, 25, 11, 29, 30,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 53, 25, 11, 29, 30,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 53, 25, 11, 29, 30,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 53, 25, 11, 29, 30,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 53, 25, 11, 29, 30,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 53, 25, 11, 29, 30,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 53, 25, 11, 29, 30,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 53, 25, 11, 29, 30,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
       device='cuda:0')

Как мы видим первые предложения совпали для каждого изображения в батче

Но они не лишены смысла:

In [25]:
for i in pred_words[0, 0, :]:
    print(vocab.idx2word[int(i)])

<start>
the
heart
size
and
pulmonary
vascularity
appear
within
normal
limits
<end>
<pad>
<pad>
<pad>
<pad>
<pad>


In [26]:
for i in pred_words[0, 1, :]:
    print(vocab.idx2word[int(i)])

<start>
the
lungs
are
free
of
focal
airspace
disease
<end>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
